# Spatial Data mining to Predict Restaurent Succeess?

In [3]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

## Introduction

According to National Restaurant Association, the restaurant industry sales hit a record high in 2015[1].   When looking at individual restaurants the number one factor for the success of a restaurant is the location and population density and demography. The study conducted by Dr. H.G. Parsa, [2] proves this statement by analyzing restaurant failures in Boulder, Colorado. But there has not been done any research with a big-data set such as Yelp academic dataset to analyze several cities with thousands of businesses and other related data which can contribute to restaurant success of failures. In this analysis we will investigate questions such as, how much of the restaurant success is accountable to its location, how urban and City planners can be benefited from this type of study or lenders or banks can assess restaurant risk factors and predict future success, how types of restaurants distributed in a city and what type of restaurant has high tendancy to success in a particular area. 
The study will be devided in to several sectoin based on the tasks needed to correctly analyse the spatial dependance of restaurants in the dataset. To measure the success of a restaurant, review_count provided in the business dataset is used. I did not use the star rating as a measure because it may not be a fair measure ,becasue there will not be a way to distinguish between rate of change of the review_count is considerd as a meauser. so only count the rewviews in 2014. Yelp is popular among many peopple by 2014 it is better to count how many review_counts happend in 2014. 

## Outline of the notebook
  
- Predict success of a restarurant based on location related predictor variables
    - Feature engineering
    - Spatial Autocorrelation Analsys
    - Spatial Autoregressoin Model Building
    - Modal Evaluation

- Hot-Spot Analysis
- Outliar Detection
- Co-location Detection

- Question Answered from Yelp Dataset
    - What type of restaurants do better in what kind of neighbourhoods ?
    - Do restaurant diversty matter in a cluster of restaurants ?

In [30]:
#load libraries
import json
import csv
import pandas as pd
from geopy.distance import vincenty
import dateutil
import datetime

In [12]:
all_review_file = "/Users/chathura/Documents/yelp_dataset_challenge_academic_dataset/yelp_academic_dataset_review.csv"
all_rw_df = pd.read_csv(all_review_file)

In [78]:
res_ids = list(All_Res_AZ['business_id'])
rev_counts_2014 = []
for res_id in res_ids:
    rev_counts_2014.append(review_count2014(res_id))

def review_count2014(res_id):
    #This function returns count of reviews happend in 2014. 
    review_count = 0
    dates = list(all_rw_df[all_rw_df['business_id'] == res_id]['date'])
    for i in dates:
        if(i.split("-")[0]=="2014"):
            review_count = review_count +1
    return review_count

All_Res_AZ['rev_counts_2014'] = rev_counts_2014

None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None


KeyboardInterrupt: 

In [14]:
#Loading Data about all businesses
business_file ="/Users/chathura/Documents/yelp_dataset_challenge_academic_dataset/yelp_academic_dataset_business.json"
bf = open(business_file,'r')
#Extract Businesses in Pheonix AZ which have high customer attraction
All_biz_AZ = [json.loads(line) for line in bf if (('AZ' in json.loads(line)['state']) & ('Restaurants' not in json.loads(line)['categories']))]
All_biz_AZ = pd.DataFrame(All_biz_AZ)
print(All_biz_AZ.shape)
zip_code=[]
for i in range(All_biz_AZ.shape[0]):
    try:
        zip_code.append(All_biz_AZ['full_address'][i].split(" ")[-1])
    except:
        zip_code.append("AZ")
All_biz_AZ['zip_code'] = zip_code
#All_biz_AZ[All_biz_AZ['zip_code']=="AZ"] # Some address does not have zip code. impute those ?
#very few restaurants in this set and not many high impact businesses as well so not worth the trouble.
All_biz_AZ = All_biz_AZ[All_biz_AZ['zip_code']!="AZ"]

#remember to reset the raw index after deleting

(17245, 15)


In [15]:
#Loading Data about all restaurants
business_file ="/Users/chathura/Documents/yelp_dataset_challenge_academic_dataset/yelp_academic_dataset_business.json"
bf = open(business_file,'r')
#Extract Businesses in Pheonix AZ which have high customer attraction
All_Res_AZ = [json.loads(line) for line in bf if (('AZ' in json.loads(line)['state']) & ('Restaurants'  in json.loads(line)['categories']))]
All_Res_AZ = pd.DataFrame(All_Res_AZ)
print(All_Res_AZ.shape)
zip_code=[]
for i in range(All_Res_AZ.shape[0]):
    try:
        zip_code.append(All_Res_AZ['full_address'][i].split(" ")[-1])
    except:
        zip_code.append("AZ")
All_Res_AZ['zip_code'] = zip_code
#All_biz_AZ[All_biz_AZ['zip_code']=="AZ"] # Some address does not have zip code. impute those ?
#very few restaurants in this set and not many high impact businesses as well so not worth the trouble.
All_Res_AZ = All_Res_AZ[All_Res_AZ['zip_code']!="AZ"]

(7985, 15)


In [16]:
All_Res_AZ

,attributes,business_id,categories,city,full_address,hours,latitude,longitude,name,neighborhoods,open,review_count,stars,state,type,zip_code
0,"{'Takes Reservations': True, 'Ambience': {'hip...",x5Mv61CnZLohZWxfCVCPTQ,"[Sandwiches, Pizza, Chicken Wings, Restaurants]",Phoenix,"2819 N Central Ave\nPhoenix, AZ 85004",{},33.479542,-112.073418,Domino's Pizza,[],True,11,3.0,AZ,business,85004
1,"{'Takes Reservations': False, 'Good for Kids':...",2ZnCITVa0abGce4gZ6RhIw,"[American (New), Sandwiches, Restaurants]",Phoenix,"1850 N Central Ave\nPhoenix, AZ 85004",{},33.468988,-112.074315,Viad Tower Restaurants,[],True,5,3.5,AZ,business,85004
2,"{'Takes Reservations': True, 'Wi-Fi': 'no', 'G...",EmzaQR5hQlF0WIl24NxAZA,"[American (New), Nightlife, Dance Clubs, Resta...",Phoenix,"132 E Washington St\nPhoenix, AZ 85004","{'Thursday': {'open': '21:00', 'close': '02:00...",33.448399,-112.071702,Sky Lounge,[],True,23,2.5,AZ,business,85004
3,"{'Takes Reservations': True, 'Ambience': {'hip...",KPoTixdjoJxSqRSEApSAGg,"[Thai, Restaurants]",Phoenix,"2631 N Central Ave\nPhoenix, AZ 85004","{'Wednesday': {'open': '11:00', 'close': '21:0...",33.477934,-112.073524,Wild Thaiger,[],True,322,4.0,AZ,business,85004
4,"{'Takes Reservations': True, 'Wi-Fi': 'no', 'O...",e5kc0CQ4R-PCCDgb274gSg,"[Mexican, Tex-Mex, Restaurants]",Phoenix,"455 N 3rd St\nSte 114\nPhoenix, AZ 85004","{'Wednesday': {'open': '11:00', 'close': '21:0...",33.452603,-112.069119,Canyon Cafe,[],True,191,3.5,AZ,business,85004
5,"{'Takes Reservations': True, 'BYOB/Corkage': '...",gBcpDl0quCLABsg5OuSFww,"[Food, Tea Rooms, Breakfast & Brunch, Sandwich...",Phoenix,"622 E Adams St\nPhoenix, AZ 85004",{},33.449539,-112.065664,Teeter House Tea Room,[],False,11,3.0,AZ,business,85004
6,"{'Drive-Thru': False, 'Waiter Service': False,...",72N6TdH5MdUWE2YhYRbyxw,"[Burgers, Restaurants]",Phoenix,"2530 N Central Ave\nPhoenix, AZ 85004",{},33.476260,-112.074248,Burger King,[],True,7,2.5,AZ,business,85004
7,"{'Takes Reservations': False, 'Waiter Service'...",eOgUKCy5-sK-leqIBiNokg,"[Cafes, Restaurants]",Phoenix,"2 N Central Ave\nPhoenix, AZ 85004","{'Tuesday': {'open': '07:00', 'close': '15:00'...",33.448491,-112.074428,Copper Square Cafe,[],True,3,3.0,AZ,business,85004
8,"{'Takes Reservations': False, 'Caters': False,...",neGAQFkBNI8-rpPBLBzNkw,"[Fast Food, Mexican, Tex-Mex, Restaurants]",Phoenix,"301 East McDowell Road\nPhoenix, AZ 85004","{'Wednesday': {'open': '07:00', 'close': '01:0...",33.465440,-112.069090,Taco Bell,[],True,8,3.5,AZ,business,85004
9,"{'Takes Reservations': True, 'Wi-Fi': 'no', 'G...",45puCRQ6Vh_IIAy7kkfFDQ,"[Bars, Restaurants, American (Traditional), Sp...",Phoenix,"24 N 2nd St\nPhoenix, AZ 85004",{},33.448465,-112.071257,Majerle's Sports Grill,[],True,170,3.5,AZ,business,85004


In [ ]:
high_Rcount_biz = All_biz_AZ[All_biz_AZ['review_count'] >= 200]
medium_Rcount_biz = All_biz_AZ[(100 < All_biz_AZ['review_count']) & (All_biz_AZ['review_count'] < 200)]
medium_Rcount_biz['R_cat']="2"
low_Rcount_biz = All_biz_AZ[All_biz_AZ['review_count'] <= 100]
high_Rcount_biz['R_cat']="3"
low_Rcount_biz['R_cat']="1"

In [ ]:
All_Res_AZ['success_cat']=pd.qcut(All_Res_AZ['review_count'],10,labels=["1","2","3","4","5","6","7","8","9","10"])
All_Res_AZ.to_csv("Arizona_restaurant_locations_with_success_cat.csv",index=False)

In [ ]:
high_Rcount_biz.to_csv("Arizona_business_locations_with_high_success_cat.csv",index=False)
medium_Rcount_biz.to_csv("Arizona_business_locations_with_medium_success_cat.csv",index=False)


In [ ]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from geopy.distance import great_circle
coordinates = low_Rcount_biz.as_matrix(columns=['longitude', 'latitude'])

In [ ]:
db = DBSCAN(eps=.01, min_samples=3).fit(coordinates)#clusters should have 10 businessess. else is error.
labels = db.labels_
num_clusters = len(set(labels)) - (1 if -1 in labels else 0)
clusters = pd.Series([coordinates[labels == i] for i in range(num_clusters)])
print('Number of clusters: %d' % num_clusters)

In [ ]:
def getCentroid(points):
    n = points.shape[0]
    sum_lon = np.sum(points[:, 1])
    sum_lat = np.sum(points[:, 0])
    return (sum_lon/n, sum_lat/n)

In [ ]:
def getNearestPoint(set_of_points, point_of_reference):
    closest_point = None
    closest_dist = None
    for point in set_of_points:
        point = (point[1], point[0])
        dist = great_circle(point_of_reference, point).meters
        if (closest_dist is None) or (dist < closest_dist):
            closest_point = point
            closest_dist = dist
    return closest_point

In [ ]:
lon = []
lat = []
for i, cluster in clusters.iteritems():
    if len(cluster) < 3:
        representative_point = (cluster[0][1], cluster[0][0])
    else:
        representative_point = getNearestPoint(cluster, getCentroid(cluster))
    lon.append(representative_point[0])
    lat.append(representative_point[1])
rs = pd.DataFrame({'lon':lon, 'lat':lat})

In [ ]:
rs['size'] = "10"
rs.to_csv("Arizona_business_locations_with_success_low_cat.csv",index=False)


In [ ]:
from IPython.display import HTML
HTML('<iframe width="100%" height="520" frameborder="0" src="https://cjgunase.cartodb.com/viz/d7f8fb6e-a5ce-11e5-8478-0e98b61680bf/embed_map" allowfullscreen webkitallowfullscreen mozallowfullscreen oallowfullscreen msallowfullscreen></iframe>')

In [ ]:
#by obervations, there is polarization 

### Predicting score based on things around it

In [ ]:
# 1. building the training set from raw data by extracting subcatagories from jsan


In [ ]:
#Extract Parking data from attribues and put in a seperate field
parking=[]
for i in range(All_Res_AZ.shape[0]):
    try:
        if(All_Res_AZ['attributes'][i]['Parking']['street']):
            parking.append("street")
        elif(All_Res_AZ['attributes'][i]['Parking']['lot']):
            parking.append("lot")
        elif(All_Res_AZ['attributes'][i]['Parking']['garage']):
            parking.append("garage")
        elif(All_Res_AZ['attributes'][i]['Parking']['validated']):
            parking.append("validated")
        elif(All_Res_AZ['attributes'][i]['Parking']['valet']):
            parking.append("valet")
        else:
            parking.append("street")
    except:
        parking.append("street")
        
All_Res_AZ['parking'] = parking

In [ ]:
All_biz_AZ = All_biz_AZ.reset_index()# Reset the index after droping some raws missing index values give errors.
#df = df.reset_index(drop=True)
All_Res_AZ = All_Res_AZ.reset_index()

In [ ]:
word = ["Blvd","Blvd.","Boulevard","St"
        ,"St.","Street","Ave","Ave.","Avenue","Dr","Drive","Ln","Parkway",
        "Pkwy","Rd","Rd.","Road","Way","Wy","Wy.","Plaze","Court","Terrace","Lp.","Lp"]

In [ ]:
address_type = []
for i in range(All_Res_AZ.shape[0]):
    if("St" in All_Res_AZ['full_address'][i]):
        address_type.append("Street")
    elif("Street" in All_Res_AZ['full_address'][i]):
        address_type.append("Street")
    elif("Blvd" in All_Res_AZ['full_address'][i]):
        address_type.append("Boulevard")
    elif("Boulevard" in All_Res_AZ['full_address'][i]):
        address_type.append("Boulevard")
    elif("Ave" in All_Res_AZ['full_address'][i]):
        address_type.append("Avenue")
    elif("Dr" in All_Res_AZ['full_address'][i]):
        address_type.append("Drive")
    elif("Drive" in All_Res_AZ['full_address'][i]):
        address_type.append("Drive")
    elif("Parkway" in All_Res_AZ['full_address'][i]):
        address_type.append("Parkway")
    elif("Pkwy" in All_Res_AZ['full_address'][i]):
        address_type.append("Parkway")
    elif("Rd" in All_Res_AZ['full_address'][i]):
        address_type.append("Road")
    elif("Road" in All_Res_AZ['full_address'][i]):
        address_type.append("Road")
    else:
        address_type.append("Other")

In [ ]:
All_Res_AZ['address_type'] = address_type

In [ ]:
import itertools
expand_res_catagoreis = list(itertools.chain(*list(All_Res_AZ['categories'])))


In [ ]:
from collections import Counter

import operator
x = Counter(expand_res_catagoreis)
sorted_x = sorted(x.items(), key=operator.itemgetter(1)).reverse()#dics can not be sorted. only a list or tuple representation

In [ ]:
sorted_x

In [ ]:
import matplotlib.pyplot as plt


D=x
plt.bar(range(len(D)), D.values(), align='center')
#plt.xticks(range(len(D)), D.keys())

plt.show()

#where each type of restaurant has high success rate.

In [ ]:
#combine dataframs to find people attracting businessess around restaurant
df1 = All_Res_AZ[['business_id','latitude','longitude','categories','review_count','zip_code']]
df1 = All_biz_AZ[['business_id','latitude','longitude','categories','review_count','zip_code']]
frames = [df1, df2]

biz_df = pd.concat(frames)

In [ ]:
def influence_biz(restaurant_id,biz_df):
    # This function returens businesses that are withing 2 mile radius from given restaurant.
    infbiz = [] # list of businesses around the restaurant
    row = All_Res_AZ[All_Res_AZ['business_id'] == restaurant_id]
    zip_code = list(row['zip_code'])#convert from data frame object to list
    lat1 = list(row['latitude'])
    lon1 = list(row['longitude'])
    biz1 = (lat1[0],lon1[0])
    nearby = biz_df[biz_df['zip_code'] == zip_code[0]] #get all businesses near by the given restaurant
    for biz in range(nearby.shape[0]):
        near_biz_id = list(nearby[0:]['business_id'])[biz]
        if(near_biz_id == restaurant_id):
            continue
        lat2 = list(biz_df[biz_df['business_id']==near_biz_id]['latitude'])
        lon2 = list(biz_df[biz_df['business_id']==near_biz_id]['longitude'])
        biz2 = (lat2[0],lon2[0])
        if(vincenty(biz1, biz2).miles < 1):
            infbiz.append(near_biz_id)
    
    #For each of the businesses in the radius calculate the following properties.
    dep_stores = 0
    restaurents=0
    shopping = 0
    hotels = 0
    arts=0
    liesure_food = 0
    medium_size = 0
    small_size = 0
    
    for biz in infbiz:
        #print (biz)
        row = biz_df[biz_df['business_id']==biz]
        score = row['review_count'].tolist()[0]
        #print(score)
        if(score > 200):
            if("Department Stores" in row['categories'].tolist()[0]):
                dep_stores=dep_stores+score
                continue
            if("Restaurants" in row['categories'].tolist()[0]):
                restaurents=restaurents+score
                continue
            if("Shopping" in row['categories'].tolist()[0]):
                shopping=shopping+score
                continue
            if("Hotels" in row['categories'].tolist()[0]):
                hotels=hotels+score
                continue
            if("Arts & Entertainment" in row['categories'].tolist()[0]):
                arts=arts+score
                continue
            if("Coffee & Tea" in row['categories'].tolist()[0]):
                liesure_food=liesure_food+score
                continue
        elif(100 < score & score < 200):
            medium_size = medium_size + score
        else:
            small_size = small_size + score
        
    inf_biz_list=[dep_stores,restaurents,shopping,hotels,arts,liesure_food,medium_size,small_size]
    return inf_biz_list

In [ ]:
list(All_biz_AZ[All_biz_AZ['business_id']=="CdqTxe171tKTOkk8T15GGA"]['categories'])

In [ ]:
list(All_Res_AZ[All_Res_AZ['business_id']=="q1nDSpmV-FdTBNj6UuZvTg"]['review_count'])